In [1]:
import pandas as pd
import re 
import spacy
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import gensim
import os
import requests
import pprint
import seaborn as sns 
import matplotlib.pyplot as plt
import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

from IPython.display import clear_output
from wordcloud import WordCloud
from collections import Counter
from joblib import dump, load
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel


pyLDAvis.enable_notebook()
#nlp = spacy.load("en_core_web_sm")

In [2]:
import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

In [3]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False, progress_bar=True)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
%%time

df = pd.read_parquet('senti_data.parquet', engine='pyarrow')
df.shape

CPU times: total: 297 ms
Wall time: 354 ms


(1650, 15)

In [5]:
df.sample(3, random_state = 42)

,date,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length,clean_text_tokens,initial_senti,sentiment_scores,average_sentiment,sentiment
482,2022-06-09,2022,6,9,3,Standigm Signs MOU with Merck Korea for AI dru...,Standigm Signs MOU with Merck Korea for AI dru...,"[standigm, signs, mou, merck, korea, ai, drug,...",5626,66,"[standigm, signs, mou, merck, korea, ai, drug,...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.350657,Positive
65,2020-06-03,2020,6,3,2,Global Artificial Intelligence in the Educatio...,Global Artificial Intelligence in the Educatio...,"[global, artificial, intelligence, education, ...",7197,207,"[global, artificial, intelligence, education, ...",1,"[0.0, 0.0, 0.4767, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",0.306625,Positive
926,2022-06-08,2022,6,8,2,Better Therapeutics gets US patent linked to m...,Better Therapeutics gets US patent linked to m...,"[better, therapeutics, get, us, patent, linked...",518,103,"[better, therapeutics, get, us, patent, linked...",0,"[0.4404, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",0.250600,Positive


### Topic Modelling on Sentiments

In [6]:
df_pos = df[df["sentiment"]=="Positive"]
df_neg = df[df["sentiment"]=="Negative"]

In [7]:
df_pos.head(2)

,date,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length,clean_text_tokens,initial_senti,sentiment_scores,average_sentiment,sentiment
0,2023-05-25,2023,5,25,3,PayPal Is Using AI to Supercharge Profits Nasdaq,PayPal Is Using AI to Supercharge Profits Nasd...,"[paypal, using, ai, supercharge, profits, nasdaq]",7090,48,"[paypal, using, ai, supercharge, profits, nasd...",0,"[0.0, 0.0, 0.0, 0.0, 0.4404, 0.0, 0.0, 0.0, 0....",0.203005,Positive
1,2023-02-24,2023,2,24,4,Facebook Parent Meta Wants to Show Its Still a...,Facebook Parent Meta Wants to Show Its Still a...,"[facebook, parent, meta, wants, show, still, b...",4689,76,"[facebook, parent, meta, wants, show, still, b...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128, 0.0...",0.179145,Positive


In [8]:
df_neg.head(5)

,date,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length,clean_text_tokens,initial_senti,sentiment_scores,average_sentiment,sentiment
11,2020-04-25,2020,4,25,5,Study reveals Artificial Intelligence can cate...,Study reveals Artificial Intelligence can cate...,"[study, reveals, artificial, intelligence, cat...",6061,101,"[study, reveals, artificial, intelligence, cat...",0,"[0.0, 0.0, 0.0, 0.4767, 0.0, -0.6597, -0.2732,...",-0.079006,Negative
22,2020-12-04,2020,12,4,4,"Google AI researchers exit sparks ethics, bias...","Google AI researchers exit sparks ethics, bias...","[google, ai, researcher, exit, spark, ethic, b...",3047,55,"[google, ai, researcher, exit, spark, ethic, b...",0,"[0.0, 0.0, 0.0, 0.0, 0.2263, 0.0, -0.1027, 0.0...",-0.024300,Negative
30,2023-03-09,2023,3,9,3,The Seinfeldinspired AI show returns to Twitch,The Seinfeldinspired AI show returns to Twitch...,"[seinfeldinspired, ai, show, return, twitch]",3229,46,"[seinfeldinspired, ai, show, return, weather, ...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.051583,Negative
67,2021-08-20,2021,8,20,4,Elon Musk introduces humanoid robot prototype ...,Elon Musk introduces humanoid robot prototype ...,"[elon, musk, introduces, humanoid, robot, prot...",5946,61,"[elon, musk, introduces, humanoid, robot, prot...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.045208,Negative
72,2020-02-18,2020,2,18,1,ECE Seminar Series: Informationtheoretic Metho...,ECE Seminar Series: Informationtheoretic Metho...,"[ece, seminar, series, informationtheoretic, m...",2702,147,"[ece, seminar, series, informationtheoretic, m...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.000340,Negative


**Postive sentiment topic modelling**

In [9]:
dictionary_text_pos = corpora.Dictionary(df_pos["clean_text_tokens"])
dictionary_text_neg = corpora.Dictionary(df_neg["clean_text_tokens"])

In [10]:
doc_term_matrix_text_pos = [dictionary_text_pos.doc2bow(doc) for doc in df_pos["clean_text_tokens"]]
doc_term_matrix_text_neg = [dictionary_text_neg.doc2bow(doc) for doc in df_neg["clean_text_tokens"]]

In [11]:
Lda = gensim.models.ldamodel.LdaModel

In [12]:
import multiprocessing
num_processors = multiprocessing.cpu_count()
num_processors

workers = num_processors

print(f'Using {workers} workers')

Using 12 workers


In [ ]:
%%time 
lda_model_pos = LdaMulticore(corpus=doc_term_matrix_text_pos,
                   id2word=dictionary_text_pos,
                   num_topics=5,
                   random_state=100,
                   passes=10,
                   eta='auto',
                   workers=workers)

print(*lda_model_pos.print_topics(num_topics=10, num_words=10), sep='\n')

**Negative sentiment topic modelling**

In [13]:
lda_model_neg = LdaMulticore(corpus=doc_term_matrix_text_neg,
                   id2word=dictionary_text_neg,
                   num_topics=5,
                   random_state=100,
                   passes=10,
                   eta='auto',
                   workers=workers)

print(*lda_model_neg.print_topics(num_topics=10, num_words=10), sep='\n')

(0, '0.007*"news" + 0.007*"ai" + 0.004*"new" + 0.004*"said" + 0.004*"technology" + 0.004*"data" + 0.003*"ha" + 0.003*"wa" + 0.003*"state" + 0.002*"year"')
(1, '0.011*"ai" + 0.006*"news" + 0.005*"cancer" + 0.003*"new" + 0.003*"health" + 0.003*"technology" + 0.003*"us" + 0.003*"data" + 0.003*"artificial" + 0.002*"intelligence"')
(2, '0.015*"ai" + 0.004*"risk" + 0.004*"news" + 0.003*"new" + 0.003*"data" + 0.003*"may" + 0.003*"system" + 0.003*"law" + 0.003*"lunit" + 0.003*"artificial"')
(3, '0.008*"news" + 0.007*"ai" + 0.005*"wa" + 0.005*"chatgpt" + 0.004*"risk" + 0.003*"company" + 0.003*"data" + 0.003*"new" + 0.003*"may" + 0.003*"said"')
(4, '0.011*"news" + 0.007*"ai" + 0.004*"cancer" + 0.004*"new" + 0.003*"one" + 0.003*"said" + 0.003*"wa" + 0.002*"model" + 0.002*"article" + 0.002*"technology"')


In [16]:
%%time

lda_display_pos = gensimvis.prepare(lda_model_pos, doc_term_matrix_text_pos, dictionary_text_pos, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display_pos)

CPU times: total: 734 ms
Wall time: 3.05 s


In [17]:
%%time

lda_display_neg = gensimvis.prepare(lda_model_neg, doc_term_matrix_text_neg, dictionary_text_neg, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display_neg)

CPU times: total: 31.2 ms
Wall time: 1.21 s


In [24]:
def extract_entities_title(text):
    import spacy
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(str(text))
    org_entities = []
    for ent in doc.ents:
        if ent.label_ in ["ORG","PERSON"]:
            org_entities.append(ent.text)
    return org_entities

def extract_entities_text(text):
    import spacy
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(str(text).lower())
    org_entities = []
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "ORG", "LOC"]:
            org_entities.append(ent.text)
    return org_entities

In [25]:
nlp = spacy.load("en_core_web_sm")

In [26]:
df_pos["ent_text"] = df_pos["clean_text"].parallel_apply(extract_entities_text)

EOFError: 

In [17]:
df_neg["ent_text"] = df_neg["clean_text"].parallel_apply(extract_entities_text)

**Positive sentiment named entities**

In [ ]:
top_entities_pos = [entity for entities in df_pos["ent_text"] for entity in entities]
Counter(top_entities_pos).most_common(15)

**Negative sentiment named entities**

In [ ]:
top_entities_neg = [entity for entities in df_neg["ent_text"] for entity in entities]
Counter(top_entities_neg).most_common(15)

### **Targeted entity sentiment analysis**

In [27]:
# Get the top 10 entities from the title
entities = [entity for entities in df_pos["ent_text"] for entity in entities]
top_entities_pos = [entity for entity, count in Counter(entities).most_common(10)]

In [28]:
rows_with_top_entities_pos = df_pos[df_pos["ent_text"].parallel_apply(lambda x: any(entity in top_entities_pos for entity in x))]
df_top_entities_pos = pd.DataFrame(rows_with_top_entities_pos)

**Positive**

In [ ]:
# Group the DataFrame by date and entity, and compute the mean sentiment score
grouped_pos = df_top_entities_pos.explode("ent_text").groupby(["date", "ent_text"])["sentiment_score"].mean().reset_index()

# Print the total number of unique entity groups
Counter(grouped_pos['ent_text']).most_common(10)

In [ ]:
df_exploded_pos = df_pos.reset_index().explode('ent_text')



search_list_pos = ['microsoft', 'google', 'europe', 'covid19', 'ibm', 'north america', 'intel', 'africa','asia','middle east']
df_filtered_pos = df_exploded_pos[df_exploded_pos['ent_text'].isin(search_list_pos)]
df_filtered_pos = df_filtered_pos.reset_index(drop=True)

# group by date and calculate mean sentiment
grouped_pos = df_filtered_pos.groupby(['year','month', 'ent_text'])['sentiment_score'].mean().reset_index()

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16, 8))

for i, elem in enumerate(search_list_pos):
    row = i//5
    col = i%5
    data = grouped_pos[grouped_pos['ent_text'] == elem]
    axes[row, col].plot(data['year'] + (data['month']-1)/12, data['sentiment_score'], label=elem)
    axes[row, col].set_xlabel('Date')
    axes[row, col].set_ylabel('Sentiment')
    axes[row, col].legend()
plt.tight_layout()
plt.show()

**Negative**

In [ ]:
entities = [entity for entities in df_neg["ent_text"] for entity in entities]
top_entities_neg = [entity for entity, count in Counter(entities).most_common(10)]

rows_with_top_entities_neg = df_neg[df_neg["ent_text"].parallel_apply(lambda x: any(entity in top_entities_neg for entity in x))]
df_top_entities_neg = pd.DataFrame(rows_with_top_entities_neg)

grouped_neg = df_top_entities_neg.explode("ent_text").groupby(["date", "ent_text"])["sentiment_score"].mean().reset_index()

Counter(grouped_neg['ent_text']).most_common(10)

In [ ]:
df_exploded_neg = df_neg.reset_index().explode('ent_text')



search_list_neg = ['covid19', 'google', 'congress', 'microsoft', 'europe', 'fcc', 'cnn', 'fbi','donald trump','joe biden']
# filter for rows with elements of interest
df_filtered_neg = df_exploded_neg[df_exploded_neg['ent_text'].isin(search_list_neg)]
df_filtered_neg = df_filtered_neg.reset_index(drop=True)

# group by date and element of interest, and calculate mean sentiment
grouped_neg = df_filtered_neg.groupby(['year','month', 'ent_text'])['sentiment_score'].mean().reset_index()

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16, 8))

for i, elem in enumerate(search_list_neg):
    row = i//5
    col = i%5
    data = grouped_neg[grouped_neg['ent_text'] == elem]
    axes[row, col].plot(data['year'] + (data['month']-1)/12, data['sentiment_score'], label=elem)
    axes[row, col].set_xlabel('Date')
    axes[row, col].set_ylabel('Sentiment')
    axes[row, col].legend()

plt.tight_layout()
plt.show()